Electric Load Control Using Virtual Buttons

In [ ]:
from cvzone.HandTrackingModule import HandDetector
from cvzone import putTextRect
from cvzone.FPS import FPS 
from cvzone import overlayPNG
from time import sleep 
import cv2
import numpy as np 
import websocket

ws = websocket.WebSocket()
ws.connect("ws://192.168.0.107:81")

fpsReader = FPS(avgCount=30)

#arduino = SerialObject("COM4", digits=1)

# Initialize the webcam to capture video
# The '2' indicates the third camera connected to your computer; '0' would usually refer to the built-in camera
cap = cv2.VideoCapture(0)

# Initialize the HandDetector class with the given parameters
detector = HandDetector(staticMode=False, maxHands=2, modelComplexity=1, detectionCon=0.6, minTrackCon=0.6)

imgRedBulb = cv2.imread(r'C:\Users\hoque\Desktop\OpenCV\VSCode\Files\RedBulb.png', cv2.IMREAD_UNCHANGED)
imgRedBulb = cv2.resize(imgRedBulb, (120,120))

imgYellowBulb = cv2.imread(r'C:\Users\hoque\Desktop\OpenCV\VSCode\Files\YelloBulb.png', cv2.IMREAD_UNCHANGED)
imgYellowBulb = cv2.resize(imgYellowBulb, (150,150))

flagRed = False 
flagYellow = False 
state1 = False 
state2 = False 

flag1, flag2, flag3 = True, True, True 

center11 = (240,80)
radius = 40 
center22 = (390,80)
color1 = (51,51,255)
color2 = (128,128,0)
point = 0,0

def checkInside(point, center, radius):
    
    distance = np.hypot(point[0] - center[0], point[1] - center[1])
    return distance <= radius


# Continuously get frames from the webcam
while True:
    # Capture each frame from the webcam
    # 'success' will be True if the frame is successfully captured, 'img' will contain the frame
    success, img = cap.read()
    H,W,_ = img.shape

    roix1, roiy1, roix2, roiy2 = 0, 0, W, int(H*0.65)
    imgroi = img[roiy1:roiy2, roix1: roix2]
    cv2.rectangle(img, (roix1, roiy1), (roix2, roiy2), (0,200,0), 2)

    # Find hands in the current frame
    # The 'draw' parameter draws landmarks and hand outlines on the image if set to True
    # The 'flipType' parameter flips the image, making it easier for some detections
    hands, imgroi = detector.findHands(imgroi, draw=True, flipType=True)

    # Check if any hands are detected
    if hands:
        # Information for the first hand detected
        hand1 = hands[0]  # Get the first hand detected
        lmList1 = hand1["lmList"]  # List of 21 landmarks for the first hand
        bbox1 = hand1["bbox"]  # Bounding box around the first hand (x,y,w,h coordinates)
        center1 = hand1['center']  # Center coordinates of the first hand
        handType1 = hand1["type"]  # Type of the first hand ("Left" or "Right")

        # Count the number of fingers up for the first hand
        fingers1 = detector.fingersUp(hand1)
        #print(f'H1 = {fingers1.count(1)}', end=" ")  # Print the count of fingers that are up

        # Calculate distance between specific landmarks on the first hand and draw it on the image
        #length, info, img = detector.findDistance(lmList1[8][0:2], lmList1[12][0:2], img, color=(255, 0, 255), scale=10)
        
        if handType1 == "Right" or handType1 == "Left": 

            point = lmList1[8][0:2]
        
            if checkInside(point, center11, radius):
                if flag1:
                    color1 = (0,255,255)
                    state1 =  not state1
                    #arduino.sendData([1])
                    data_to_send = '1'  # Replace '5' with '3' to turn off the LED
                    ws.send(data_to_send)
                    flag1=False 
                    flagRed = True 

            else:
                 flag1=True 
                 if state1:
                    color1=(0,210,0)
                 else: 
                    color1=(51,51,255)

            if checkInside(point, center22, radius):
                if flag2:
                    color2 = (0,255,255)
                    state2 = not state2 
                    #arduino.sendData([2])
                    data_to_send = '2'  # Replace '5' with '3' to turn off the LED
                    ws.send(data_to_send)
                    flag2= False 
                    flagYellow = True
                    flag3 = True 
            else:
                 flag2 = True 
                 if state2:
                    color2=(0,210,0)
                 else: 
                    color2=(128,128,0)
                

        # if handType1 == "Left": 
        #     if fingers1.count(1) == 0 and flag3:
        #         #arduino.sendData([0])
        #         data_to_send = '0'  # Replace '5' with '3' to turn off the LED
                
        #         flagRed = False
        #         flagYellow = False 
               

          

        # Check if a second hand is detected
        if len(hands) == 2:
            # Information for the second hand
            hand2 = hands[1]
            lmList2 = hand2["lmList"]
            bbox2 = hand2["bbox"]
            center2 = hand2['center']
            handType2 = hand2["type"]

            # Count the number of fingers up for the second hand
            fingers2 = detector.fingersUp(hand2)
            #print(f'H2 = {fingers2.count(1)}', end=" ")

            # Calculate distance between the index fingers of both hands and draw it on the image
            #length, info, img = detector.findDistance(lmList1[8][0:2], lmList2[8][0:2], img, color=(255, 0, 0), scale=10)

        #print(" ")  # New line for better readability of the printed output

    cv2.rectangle(img, (center11[0]-50, center11[1]-50), (center11[0]+50, center11[1]+50), (139,61,72), cv2.FILLED)
    cv2.circle(img, center11, radius, color1, cv2.FILLED)
    cv2.rectangle(img, (center22[0]-50, center22[1]-50), (center22[0]+50, center22[1]+50), (139,61,72), cv2.FILLED)
    cv2.circle(img, center22, radius, color2, cv2.FILLED)
    cv2.putText(img, "B1",(223,90), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.putText(img, "B2",(371,88), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    putTextRect(img, text="VIRTUAL BUTTONS", pos=(int(200), 165), scale=1.6, thickness=2, colorR=(127,0,255), offset=8, border=3, colorB=(0,0,0) )

    #fpsReader.update returns the current FPS and the updated image
    cv2.circle(img, point, 7, (112,25,25), cv2.FILLED)

    if state1: 
         data_to_send=1

    if state1: 
            overlayPNG(img, imgRedBulb, [int(W*0.01), 30])
            putTextRect(img, text="BULB1 ON", pos=(int(W*0.015), 200), scale=1.3, thickness=2, colorR=(0,0,210), offset=8, border=3, colorB=(0,0,0) )
    if state2: 
            overlayPNG(img, imgYellowBulb, [int(W-W*0.20), 30])
            putTextRect(img, text="BULB2 ON", pos=(int(W-W*0.18), 200), scale=1.3, thickness=2, colorR=(0,150,250), offset=8, border=3, colorB=(0,0,0) )

    #fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(10, 470),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)
    # Display the image in a window
    cv2.imshow("Image", img)

    # Wait for 1 millisecond to check for any user input, keeping the window open
    if cv2.waitKey(1) & 0xFF == ord('q'):
       break
cap.release()
cv2.destroyAllWindows()